In [71]:
import numpy as np
import pandas as pd
import datetime as dt
pd.set_option('display.max_rows', 1000)
import hvplot.pandas
import holoviews as hv
from math import log

pd.options.mode.chained_assignment = None  # default='warn'

<a id='top'></a>
Suggested Reading:
[Coronavirus Interview with Larry Brilliant](https://www.wired.com/story/coronavirus-interview-larry-brilliant-smallpox-epidemiologist/?utm_source=pocket-newtab)

[Confirmed Cases: USA](#Confirmed_Cases_USA)  
[Confirmed Cases: Germany](#Confirmed_Cases_Germany)  
[Confirmed Cases: Country Comparison](#Confirmed_Cases_Country_Comparison)  
[Confirmed Cases: U.S. States Comparison](#Confirmed_Cases_US_Comparison)  
[Deaths: Country Comparison](#Deaths_Country_Comparison)  
[Deaths: U.S States Comparison](#Deaths_US_Comparison)  
[Mortality](#Mortality)

**NOTE: Move cursor over the plots to see actual data points**



# Daily Analysis of COVID-19 Data from John Hopkins University

In [2]:
print('\t\tUpdated every 6 hours. Last update:',dt.datetime.now().strftime('%Y-%m-%d %H:%M UTC'))

		Updated every 6 hours. Last update: 2020-03-21 16:58 UTC


Josef Kellndorfer, Ph.D., Earth Big Data, LLC, Richard Signell, Ph.D., USGS

These plots show the daily status of COVID-19 cases as reported by John Hopkins University. Please use freely to look at daily changes and trends. Keep in mind that data are changing frequently as more Covid-19 tests become available globally. We chose to plot **totals and** numbers **normalized by population** (expressed as per 100,000). Also, it is advantageous to plot case totals (confirmed infections, deaths, and recovered) in **logarithmic scale** where trends and parallels between countries become more obvious. Note, that a straight line trending upwards in logarithmic scale indicates exponential increase! Taking a close look at the plots, one will discern differences and similarities, and that for the most part initial stages are **similar in all countries with a time lag**. What to look out for is whether the measures taken by countries, foremost social distancing show the desired effects of slowing and eventually reversing the exponential upwards trends. The first set of plots looks at confirmed infections, the bottom set of plots looks at confirmed deaths and mortality which may be somewhat more reliable with respect to an impact for a country while tests are rolled out in larger numbers.

This is a work in progress, stay tuned. 

You can get the notebook underlying this work at:
[https://github.com/EarthBigData/covid19](https://github.com/EarthBigData/covid19)

For questions: Please use the email webform at [earthbigdata.com](http://earthbigdata.com)

## Data Sources

### John Hopkins University daily updated COVID-19 data
COVID-19 confirmed cases, deaths and recovered cases data are streamed from the [The Center for Systems Science and Engineering (CSSE)](https://systems.jhu.edu) at Johns Hopkins University. The CCSE COVID-19 [GitHub Repo](https://github.com/CSSEGISandData/COVID-19) has more information about these data and their sources.

### UN Population Data
We obtain the Population data from UN statistics.  [UN Population Data Sets](https://population.un.org/wpp/Download/Standard/Population/) have more information about these data and their sources.

### US Population Data
US population data ar obtained from US Census statistics.  [US Population Data Sets](http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv?#) have more information about these data and their sources.

## Interacting with the plots

You can use the control buttons to interact with the plots, e.g. zoom in/out or also hover over the data points to get a detailed number.



In [72]:
?hv.extension.logo

Type:        bool
String form: True
Docstring:  
bool(x) -> bool

Returns True when the argument x is true, False otherwise.
The builtins True and False are the only two instances of the class bool.
The class bool is a subclass of the class int, and cannot be subclassed.


In [3]:
# World population
# pop='https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_TotalPopulationBySex.csv'
# We use a locally downloaded dataset from the link above
pop= '/s/data//WPP2019_TotalPopulationBySex.csv'

popdf = pd.read_csv(pop)
popdf.rename(columns={'Location': 'country','PopTotal':'population'}, inplace = True)

#popdf[(popdf.Time==2019) & (popdf.country=='China')].head(20)

popdf2=popdf[popdf.Time==2019]
#[ x for x in popdf2.country.values if x.find('Korea') > -1]

# Necessary adjustmens of names so we can link the population and jhu data sets
popdf2.loc[popdf2.country=='United States of America','country']='US'
popdf2.loc[popdf2.country=='Iran (Islamic Republic of)','country']='Iran'
popdf2.loc[popdf2.country=='Republic of Korea','country']='Korea, South'

In [4]:
# US Population
#popus='http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv?#'
popus='/s/data/nst-est2019-alldata.csv'
popusdf = pd.read_csv(popus)
popusdf['POPESTIMATE2019']
popusdf.rename(columns={'NAME': 'state','POPESTIMATE2019':'population'}, inplace = True)

In [5]:
# Sources for COVID-19 
# c = confirmed cases
# d = deaths
# r = recovered

cv19_c ='https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
cv19_d ='https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
cv19_r ='https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'

<a id='Confirmed_Cases_USA'></a>
## Confirmed Cases: Status for United States
[back to top](#top)

In [20]:
df = pd.read_csv(cv19_c)
df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df = df[(df.country=='US') & (df.state!='Diamond Princess') & 
        (df.state!='Grand Princess')].reset_index(drop=True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))
state = df.state.str.split(',').apply(lambda x: x[-1].strip())
county = df.state.str.split(',').apply(lambda x: x[0].strip())
county[~df.state.str.contains(',')] = None
df.state = state
df.insert(0, 'county', county)
#df.head()

logscale=False
title='Confirmed cases of COVID-19 in the USA. Status: {}'.format(df.columns[-1].date())

s = df.select_dtypes(np.int).sum()
upperlim=s[-1]*1.2
ylim=(1,upperlim)
s.name = 'USA'

logscale=False
title='Confirmed cases of COVID-19 in {}, Status: {}'.format(s.name,df.columns[-1].date())
opts = {'legend':True, 'logy': logscale, 'grid': True,'title': title, 'padding':0.1,'ylim':ylim,
        'ylabel':'Number of Confirmed Cases','xlabel':'Date','width':700}
lines = s.hvplot(**opts) 
dots  = s.hvplot.scatter(**opts)
layoutlin = (lines * dots).opts(active_tools=['box_zoom'])

logscale=True
title='Confirmed cases of COVID-19 in {}, Status: {}, Logarithmic Scale'.format(s.name,df.columns[-1].date())
opts = {'legend':True, 'logy': logscale, 'grid': True,'title': title, 'padding':0.1,'ylim':ylim,
        'ylabel':'Number of Confirmed Cases','xlabel':'Date','width':700}
lines = s.hvplot(**opts) 
dots  = s.hvplot.scatter(**opts)
layoutlog = (lines * dots).opts(active_tools=['box_zoom'])

layout=hv.Layout([layoutlin,layoutlog]).opts(shared_axes=False,merge_tools=False)
#layout.toolbar.active_drag = None
layout.cols(1)

:Layout
   .Overlay.USA.I  :Overlay
      .Curve.USA   :Curve   [index]   (USA)
      .Scatter.USA :Scatter   [index]   (USA)
   .Overlay.USA.II :Overlay
      .Curve.USA   :Curve   [index]   (USA)
      .Scatter.USA :Scatter   [index]   (USA)

<a id='Confirmed_Cases_Germany'></a>
## Confirmed Cases: Status for Germany
[back to top](#top)

In [21]:
df = pd.read_csv(cv19_c)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))
df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df.state='Germany'
df=df[(df.country=='Germany')].reset_index(drop=True)
state = df.state.str.split(',').apply(lambda x: x[-1].strip())
county = df.state.str.split(',').apply(lambda x: x[0].strip())
county[~df.state.str.contains(',')] = None
df.state = state
df.insert(0, 'county', county)
# df.head()

s = df.select_dtypes(np.int).sum()
upperlim=s[-1]*1.2
ylim=(1,upperlim)

s.name = 'Germany'

logscale=False
title='Confirmed cases of COVID-19 in Germany, Status: {}'.format(df.columns[-1].date())
# opts = {'legend':True, 'logy': logscale, 'grid': True, 'width': 350, 'height': 400,
#         'title': title, 'padding':0.1,'ylim':ylim,'ylabel':'Number of Confirmed Cases','xlabel':'Date'}
opts = {'legend':True, 'logy': logscale, 'grid': True,'title': title, 'padding':0.1,'ylim':ylim,
        'ylabel':'Number of Confirmed Cases','xlabel':'Date','width':700}
lines = s.hvplot(**opts) 
dots  = s.hvplot.scatter(**opts)
layoutlin = (lines * dots).opts(active_tools=['box_zoom'])

logscale=True
title='Confirmed cases of COVID-19 in Germany, Status: {}, Logarithmic Scale'.format(df.columns[-1].date())
# opts = {'legend':True, 'logy': logscale, 'grid': True, 'width': 350, 'height': 400,
#         'title': title, 'padding':0.1,'ylim':ylim,'ylabel':'Number of Confirmed Cases','xlabel':'Date'}
opts = {'legend':True, 'logy': logscale, 'grid': True,'title': title, 'padding':0.1,'ylim':ylim,
        'ylabel':'Number of Confirmed Cases','xlabel':'Date','width':700}
lines = s.hvplot(**opts) 
dots  = s.hvplot.scatter(**opts)
layoutlog = (lines * dots).opts(active_tools=['box_zoom'])

layout=hv.Layout([layoutlin,layoutlog]).opts(shared_axes=False,merge_tools=False)
layout.cols(1)

:Layout
   .Overlay.Germany.I  :Overlay
      .Curve.Germany   :Curve   [index]   (Germany)
      .Scatter.Germany :Scatter   [index]   (Germany)
   .Overlay.Germany.II :Overlay
      .Curve.Germany   :Curve   [index]   (Germany)
      .Scatter.Germany :Scatter   [index]   (Germany)

<a id='Confirmed_Cases_Country_Comparison'></a>
##  Confirmed Cases: Country Comparison
[back to top](#top)

In [60]:
countries=('China','Korea, South','Japan','Italy','Iran','Spain','Germany','Austria','France','US','Mexico','Canada','Brazil','India')

df = pd.read_csv(cv19_c)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))
df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)

df = df[[x in countries for x in df.country.values]]

def uplim(df,pdf,normalize=False,extend=1.2):  
    smax=0
    for i in df.country:
        s = df[df.country==i].select_dtypes(np.int).sum()[-1]
        if normalize:
            s = s/(pdf[(pdf.country==i) & (pdf.Time==2019)].population.values[0]*1000)*100000
        if s > smax:
            smax=s   
    return smax*extend,log(smax*extend) 

def country(name='Germany',normalize_by_population=False,logy=False):
    country = df[(df.country==name)]
    s = country.select_dtypes(np.int).sum()
    if normalize_by_population:
        ulim,ulimlog=uplim(country,popdf2)
        s = s/(popdf2[(popdf2.country==name) & (popdf2.Time==2019)].population.values[0]*1000)*100000
#         upperlim=s[-1]*1.2
#         ylim=(0.00001,upperlim)
#         print(ylim)
#     else:
#         upperlim=s[-1]*1.2
#         ylim=(1,upperlim)
#     opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,
#         'title': f'Confirmed cases of COVID-19 in {name}', 'padding':0.1,
#         'ylim':ylim}
    opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,'padding':0.1}
    s.name = name
    lines = s.hvplot(**opts)
    dots = s.hvplot.scatter(**opts)
    hstate = (lines * dots)
    return hstate

ulim,ulimlog=uplim(df,popdf2)
nulim,nulimlog=uplim(df,popdf2,True)

lo=[]
for normalize_by_population in [False,True]:
    for logscale in [False, True]:
    
        if normalize_by_population:
            ylabel='Number of cases normalized by population [per 100,000]'
            title_format='COVID-19 Confirmed Cases: Normalized by Population, Status: {}'.format(df.columns[-1].date())
            ylim=(0.00001,nulim)
        else:
            ylim=(1,ulim)
            ylabel='Total Number of Cases [Count]'
            title_format='COVID-19 Confirmed Cases: Total Count, Status: {}'.format(df.columns[-1].date())

        if logscale:
            title_format+=', Logarithmic Scale'

        ol=[]
        for i in countries:
            ol.append(country(i,normalize_by_population,logscale))

        overlay=hv.Overlay(ol)
        lo.append(overlay.opts(height=600,width=780,title_format=title_format,ylabel=ylabel,
                               xlabel='Date',xrotation=45,ylim=ylim,logy=logscale,active_tools=['box_zoom'],toolbar='right'))
layout=hv.Layout(lo).opts(shared_axes=False,merge_tools=False)
layout.cols(1)

:Layout
   .Overlay.I   :Overlay
      .Curve.China               :Curve   [index]   (China)
      .Scatter.China             :Scatter   [index]   (China)
      .Curve.Korea_comma_South   :Curve   [index]   (Korea, South)
      .Scatter.Korea_comma_South :Scatter   [index]   (Korea, South)
      .Curve.Japan               :Curve   [index]   (Japan)
      .Scatter.Japan             :Scatter   [index]   (Japan)
      .Curve.Italy               :Curve   [index]   (Italy)
      .Scatter.Italy             :Scatter   [index]   (Italy)
      .Curve.Iran                :Curve   [index]   (Iran)
      .Scatter.Iran              :Scatter   [index]   (Iran)
      .Curve.Spain               :Curve   [index]   (Spain)
      .Scatter.Spain             :Scatter   [index]   (Spain)
      .Curve.Germany             :Curve   [index]   (Germany)
      .Scatter.Germany           :Scatter   [index]   (Germany)
      .Curve.Austria             :Curve   [index]   (Austria)
      .Scatter.Austria           :Scatter   [index]   (Austria)
      .Curve.France              :Curve   [index]   (France)
      .Scatter.France            :Scatter   [index]   (France)
      .Curve.US                  :Curve   [index]   (US)
      .Scatter.US                :Scatter   [index]   (US)
      .Curve.Mexico              :Curve   [index]   (Mexico)
      .Scatter.Mexico            :Scatter   [index]   (Mexico)
      .Curve.Canada              :Curve   [index]   (Canada)
      .Scatter.Canada            :Scatter   [index]   (Canada)
      .Curve.Brazil              :Curve   [index]   (Brazil)
      .Scatter.Brazil            :Scatter   [index]   (Brazil)
      .Curve.India               :Curve   [index]   (India)
      .Scatter.India             :Scatter   [index]   (India)
   .Overlay.II  :Overlay
      .Curve.China               :Curve   [index]   (China)
      .Scatter.China             :Scatter   [index]   (China)
      .Curve.Korea_comma_South   :Curve   [index]   (Korea, South)
      .Scatter.Korea_comma_South :Scatter   [index]   (Korea, South)
      .Curve.Japan               :Curve   [index]   (Japan)
      .Scatter.Japan             :Scatter   [index]   (Japan)
      .Curve.Italy               :Curve   [index]   (Italy)
      .Scatter.Italy             :Scatter   [index]   (Italy)
      .Curve.Iran                :Curve   [index]   (Iran)
      .Scatter.Iran              :Scatter   [index]   (Iran)
      .Curve.Spain               :Curve   [index]   (Spain)
      .Scatter.Spain             :Scatter   [index]   (Spain)
      .Curve.Germany             :Curve   [index]   (Germany)
      .Scatter.Germany           :Scatter   [index]   (Germany)
      .Curve.Austria             :Curve   [index]   (Austria)
      .Scatter.Austria           :Scatter   [index]   (Austria)
      .Curve.France              :Curve   [index]   (France)
      .Scatter.France            :Scatter   [index]   (France)
      .Curve.US                  :Curve   [index]   (US)
      .Scatter.US                :Scatter   [index]   (US)
      .Curve.Mexico              :Curve   [index]   (Mexico)
      .Scatter.Mexico            :Scatter   [index]   (Mexico)
      .Curve.Canada              :Curve   [index]   (Canada)
      .Scatter.Canada            :Scatter   [index]   (Canada)
      .Curve.Brazil              :Curve   [index]   (Brazil)
      .Scatter.Brazil            :Scatter   [index]   (Brazil)
      .Curve.India               :Curve   [index]   (India)
      .Scatter.India             :Scatter   [index]   (India)
   .Overlay.III :Overlay
      .Curve.China               :Curve   [index]   (China)
      .Scatter.China             :Scatter   [index]   (China)
      .Curve.Korea_comma_South   :Curve   [index]   (Korea, South)
      .Scatter.Korea_comma_South :Scatter   [index]   (Korea, South)
      .Curve.Japan               :Curve   [index]   (Japan)
      .Scatter.Japan             :Scatter   [index]   (Japan)
      .Curve.Italy               :Curve   [index]   (Italy)
      .Scatter.It

<a id='Confirmed_Cases_US_Comparison'></a>
## Confirmed Cases: U.S. States Comparison
[back to top](#top)

In [67]:
states=(('Massachusetts','MA'),('Connecticut','CT'),('Washington','WA'),('Texas','TX'),('California','CA'),('New York','NY'),('Georgia','GA'),('Illinois','IL'),('Michigan','MI'))

df = pd.read_csv(cv19_c)
df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df = df[(df.country=='US') & (df.state!='Diamond Princess') & 
        (df.state!='Grand Princess')].reset_index(drop=True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))
state = df.state.str.split(',').apply(lambda x: x[-1].strip())
county = df.state.str.split(',').apply(lambda x: x[0].strip())
county[~df.state.str.contains(',')] = None
df.state = state
df.insert(0, 'county', county)


snames=[x[0] for x in states]
scodes=[x[1] for x in states]
df = df[[x in snames or x in scodes for x in df.state.values]]

def uplim(states,df,pdf,normalize=False,extend=1.2):  
    smax=0
    for i in states:
        name,code=i
        state = df[(df.state==name) | (df.state==code)]
        s = state.select_dtypes(np.int).sum()[-1]
        if normalize:
            s = s/pdf[pdf.state==name].population.values[0]*100000
        if s > smax:
            smax=s   
    return smax*extend,log(smax*extend) 

def state(name='Massachusetts',code='MA',normalize_by_population=False,logy=False):
    state = df[(df.state==name) | (df.state==code)]
    s = state.select_dtypes(np.int).sum()
    if normalize_by_population:
        s = s/popusdf[popusdf.state==name].population.values[0]*100000
    opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,'padding':0.1}
    s.name = name
    lines = s.hvplot(**opts)
    dots = s.hvplot.scatter(**opts)
    hstate = lines * dots
    return hstate

sd='2020-02-15'
ed=(df.columns[-1]+dt.timedelta(1)).strftime('%Y-%m-%d')
xlim=(pd.date_range(sd,ed)[0],pd.date_range(sd,ed)[-1])

ulim,ulimlog=uplim(states,df,popusdf)
nulim,nulimlog=uplim(states,df,popusdf,True)

lo=[]
for normalize_by_population in [False,True]:
    for logscale in [False, True]:
    
        if normalize_by_population:
            ylim=(0.001,nulim)
            ylabel='Number of cases normalized by population [per 100,000]'
            title_format='U.S. COVID-19 Confirmed Cases: Normalized by Population, Status: {}'.format(df.columns[-1].date())
        else:
            ylim=(1,ulim)
            ylabel='U.S. Total Number of Cases [Count]'
            title_format='U.S. COVID-19 Confirmed Cases: Total Count, Status: {}'.format(df.columns[-1].date())
        ol=[]
        for i in states:
            ol.append(state(*i,normalize_by_population,logscale))

        if logscale:
            title_format+=', Log. Scale'

        overlay=hv.Overlay(ol)
        lo.append(overlay.opts(height=600,width=780,title_format=title_format, 
                               ylim=ylim,xlim=xlim,ylabel=ylabel,xlabel='Date',
                               xrotation=45,active_tools=['box_zoom'],toolbar='right'))
layout=hv.Layout(lo).opts(shared_axes=False,merge_tools=False)
layout.cols(1)

:Layout
   .Overlay.I   :Overlay
      .Curve.Massachusetts   :Curve   [index]   (Massachusetts)
      .Scatter.Massachusetts :Scatter   [index]   (Massachusetts)
      .Curve.Connecticut     :Curve   [index]   (Connecticut)
      .Scatter.Connecticut   :Scatter   [index]   (Connecticut)
      .Curve.Washington      :Curve   [index]   (Washington)
      .Scatter.Washington    :Scatter   [index]   (Washington)
      .Curve.Texas           :Curve   [index]   (Texas)
      .Scatter.Texas         :Scatter   [index]   (Texas)
      .Curve.California      :Curve   [index]   (California)
      .Scatter.California    :Scatter   [index]   (California)
      .Curve.New_York        :Curve   [index]   (New York)
      .Scatter.New_York      :Scatter   [index]   (New York)
      .Curve.Georgia         :Curve   [index]   (Georgia)
      .Scatter.Georgia       :Scatter   [index]   (Georgia)
      .Curve.Illinois        :Curve   [index]   (Illinois)
      .Scatter.Illinois      :Scatter   [index]   (Illinois)
      .Curve.Michigan        :Curve   [index]   (Michigan)
      .Scatter.Michigan      :Scatter   [index]   (Michigan)
   .Overlay.II  :Overlay
      .Curve.Massachusetts   :Curve   [index]   (Massachusetts)
      .Scatter.Massachusetts :Scatter   [index]   (Massachusetts)
      .Curve.Connecticut     :Curve   [index]   (Connecticut)
      .Scatter.Connecticut   :Scatter   [index]   (Connecticut)
      .Curve.Washington      :Curve   [index]   (Washington)
      .Scatter.Washington    :Scatter   [index]   (Washington)
      .Curve.Texas           :Curve   [index]   (Texas)
      .Scatter.Texas         :Scatter   [index]   (Texas)
      .Curve.California      :Curve   [index]   (California)
      .Scatter.California    :Scatter   [index]   (California)
      .Curve.New_York        :Curve   [index]   (New York)
      .Scatter.New_York      :Scatter   [index]   (New York)
      .Curve.Georgia         :Curve   [index]   (Georgia)
      .Scatter.Georgia       :Scatter   [index]   (Georgia)
      .Curve.Illinois        :Curve   [index]   (Illinois)
      .Scatter.Illinois      :Scatter   [index]   (Illinois)
      .Curve.Michigan        :Curve   [index]   (Michigan)
      .Scatter.Michigan      :Scatter   [index]   (Michigan)
   .Overlay.III :Overlay
      .Curve.Massachusetts   :Curve   [index]   (Massachusetts)
      .Scatter.Massachusetts :Scatter   [index]   (Massachusetts)
      .Curve.Connecticut     :Curve   [index]   (Connecticut)
      .Scatter.Connecticut   :Scatter   [index]   (Connecticut)
      .Curve.Washington      :Curve   [index]   (Washington)
      .Scatter.Washington    :Scatter   [index]   (Washington)
      .Curve.Texas           :Curve   [index]   (Texas)
      .Scatter.Texas         :Scatter   [index]   (Texas)
      .Curve.California      :Curve   [index]   (California)
      .Scatter.California    :Scatter   [index]   (California)
      .Curve.New_York        :Curve   [index]   (New York)
      .Scatter.New_York      :Scatter   [index]   (New York)
      .Curve.Georgia         :Curve   [index]   (Georgia)
      .Scatter.Georgia       :Scatter   [index]   (Georgia)
      .Curve.Illinois        :Curve   [index]   (Illinois)
      .Scatter.Illinois      :Scatter   [index]   (Illinois)
      .Curve.Michigan        :Curve   [index]   (Michigan)
      .Scatter.Michigan      :Scatter   [index]   (Michigan)
   .Overlay.IV  :Overlay
      .Curve.Massachusetts   :Curve   [index]   (Massachusetts)
      .Scatter.Massachusetts :Scatter   [index]   (Massachusetts)
      .Curve.Connecticut     :Curve   [index]   (Connecticut)
      .Scatter.Connecticut   :Scatter   [index]   (Connecticut)
      .Curve.Washington      :Curve   [index]   (Washington)
      .Scatter.Washington    :Scatter   [index]   (Washington)
      .Curve.Texas           :Curve   [index]   (Texas)
      .Scatter.Texas         :Scatter   [index]   (Texas)
      .Curve.California      :Curve   [index]   (California)
      .Scatter.California    :Scatter   [index]  

<a id='Deaths_Country_Comparison'></a>
## Deaths Country Comparison
[back to top](#top)

In [62]:
countries=('China','Korea, South','Japan','Italy','Iran','Spain','Germany','Austria','France','US','Mexico','Canada','Brazil','India')

df = pd.read_csv(cv19_d)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))
df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)

df = df[[x in countries for x in df.country.values]]

def uplim(df,pdf,normalize=False,extend=1.2):  
    smax=0
    for i in df.country:
        s = df[df.country==i].select_dtypes(np.int).sum()[-1]
        if normalize:
            s = s/(pdf[(pdf.country==i) & (pdf.Time==2019)].population.values[0]*1000)*100000
        if s > smax:
            smax=s   
    return smax*extend,log(smax*extend) 

def country(name='Germany',normalize_by_population=False,logy=False):
    country = df[(df.country==name)]
    s = country.select_dtypes(np.int).sum()
    if normalize_by_population:
        ulim,ulimlog=uplim(country,popdf2)
        s = s/(popdf2[(popdf2.country==name) & (popdf2.Time==2019)].population.values[0]*1000)*100000
#         upperlim=s[-1]*1.2
#         ylim=(0.00001,upperlim)
#         print(ylim)
#     else:
#         upperlim=s[-1]*1.2
#         ylim=(1,upperlim)
#     opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,
#         'title': f'Confirmed cases of COVID-19 in {name}', 'padding':0.1,
#         'ylim':ylim}
    opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,'padding':0.1}
    s.name = name
    lines = s.hvplot(**opts)
    dots = s.hvplot.scatter(**opts)
    hstate = lines * dots
    return hstate

ulim,ulimlog=uplim(df,popdf2)
nulim,nulimlog=uplim(df,popdf2,True)

lo=[]
for normalize_by_population in [False,True]:
    for logscale in [False, True]:
    
        if normalize_by_population:
            ylabel='Number of Deaths normalized by population [per 100,000]'
            title_format='COVID-19 Deaths: Normalized by Population, Status: {}'.format(df.columns[-1].date())
            ylim=(0.00001,nulim)
        else:
            ylim=(1,ulim)
            ylabel='Total Number of Deaths [Count]'
            title_format='COVID-19 Deaths: Total Count, Status: {}'.format(df.columns[-1].date())

        if logscale:
            title_format+=', Logarithmic Scale'

        ol=[]
        for i in countries:
            ol.append(country(i,normalize_by_population,logscale))

        overlay=hv.Overlay(ol)
        lo.append(overlay.opts(height=600,width=780,title_format=title_format,ylabel=ylabel,
                               xlabel='Date',xrotation=45,ylim=ylim,logy=logscale,active_tools=['box_zoom'],toolbar='right'))
layout=hv.Layout(lo).opts(shared_axes=False,merge_tools=False)
layout.cols(1)

:Layout
   .Overlay.I   :Overlay
      .Curve.China               :Curve   [index]   (China)
      .Scatter.China             :Scatter   [index]   (China)
      .Curve.Korea_comma_South   :Curve   [index]   (Korea, South)
      .Scatter.Korea_comma_South :Scatter   [index]   (Korea, South)
      .Curve.Japan               :Curve   [index]   (Japan)
      .Scatter.Japan             :Scatter   [index]   (Japan)
      .Curve.Italy               :Curve   [index]   (Italy)
      .Scatter.Italy             :Scatter   [index]   (Italy)
      .Curve.Iran                :Curve   [index]   (Iran)
      .Scatter.Iran              :Scatter   [index]   (Iran)
      .Curve.Spain               :Curve   [index]   (Spain)
      .Scatter.Spain             :Scatter   [index]   (Spain)
      .Curve.Germany             :Curve   [index]   (Germany)
      .Scatter.Germany           :Scatter   [index]   (Germany)
      .Curve.Austria             :Curve   [index]   (Austria)
      .Scatter.Austria           :Scatter   [index]   (Austria)
      .Curve.France              :Curve   [index]   (France)
      .Scatter.France            :Scatter   [index]   (France)
      .Curve.US                  :Curve   [index]   (US)
      .Scatter.US                :Scatter   [index]   (US)
      .Curve.Mexico              :Curve   [index]   (Mexico)
      .Scatter.Mexico            :Scatter   [index]   (Mexico)
      .Curve.Canada              :Curve   [index]   (Canada)
      .Scatter.Canada            :Scatter   [index]   (Canada)
      .Curve.Brazil              :Curve   [index]   (Brazil)
      .Scatter.Brazil            :Scatter   [index]   (Brazil)
      .Curve.India               :Curve   [index]   (India)
      .Scatter.India             :Scatter   [index]   (India)
   .Overlay.II  :Overlay
      .Curve.China               :Curve   [index]   (China)
      .Scatter.China             :Scatter   [index]   (China)
      .Curve.Korea_comma_South   :Curve   [index]   (Korea, South)
      .Scatter.Korea_comma_South :Scatter   [index]   (Korea, South)
      .Curve.Japan               :Curve   [index]   (Japan)
      .Scatter.Japan             :Scatter   [index]   (Japan)
      .Curve.Italy               :Curve   [index]   (Italy)
      .Scatter.Italy             :Scatter   [index]   (Italy)
      .Curve.Iran                :Curve   [index]   (Iran)
      .Scatter.Iran              :Scatter   [index]   (Iran)
      .Curve.Spain               :Curve   [index]   (Spain)
      .Scatter.Spain             :Scatter   [index]   (Spain)
      .Curve.Germany             :Curve   [index]   (Germany)
      .Scatter.Germany           :Scatter   [index]   (Germany)
      .Curve.Austria             :Curve   [index]   (Austria)
      .Scatter.Austria           :Scatter   [index]   (Austria)
      .Curve.France              :Curve   [index]   (France)
      .Scatter.France            :Scatter   [index]   (France)
      .Curve.US                  :Curve   [index]   (US)
      .Scatter.US                :Scatter   [index]   (US)
      .Curve.Mexico              :Curve   [index]   (Mexico)
      .Scatter.Mexico            :Scatter   [index]   (Mexico)
      .Curve.Canada              :Curve   [index]   (Canada)
      .Scatter.Canada            :Scatter   [index]   (Canada)
      .Curve.Brazil              :Curve   [index]   (Brazil)
      .Scatter.Brazil            :Scatter   [index]   (Brazil)
      .Curve.India               :Curve   [index]   (India)
      .Scatter.India             :Scatter   [index]   (India)
   .Overlay.III :Overlay
      .Curve.China               :Curve   [index]   (China)
      .Scatter.China             :Scatter   [index]   (China)
      .Curve.Korea_comma_South   :Curve   [index]   (Korea, South)
      .Scatter.Korea_comma_South :Scatter   [index]   (Korea, South)
      .Curve.Japan               :Curve   [index]   (Japan)
      .Scatter.Japan             :Scatter   [index]   (Japan)
      .Curve.Italy               :Curve   [index]   (Italy)
      .Scatter.It

<a id='Deaths_US_Comparison'></a>
## U.S.Deaths
[back to top](#top)

In [69]:
states=(('Massachusetts','MA'),('Connecticut','CT'),('Washington','WA'),('Texas','TX'),('California','CA'),('New York','NY'),('Georgia','GA'),('Illinois','IL'),('Michigan','MI'))

df = pd.read_csv(cv19_d)
df.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df = df[(df.country=='US') & (df.state!='Diamond Princess') & 
        (df.state!='Grand Princess')].reset_index(drop=True)
df.columns = df.columns[0:4].append(pd.to_datetime(df.columns[4:]))
state = df.state.str.split(',').apply(lambda x: x[-1].strip())
county = df.state.str.split(',').apply(lambda x: x[0].strip())
county[~df.state.str.contains(',')] = None
df.state = state
df.insert(0, 'county', county)


snames=[x[0] for x in states]
scodes=[x[1] for x in states]
df = df[[x in snames or x in scodes for x in df.state.values]]

def uplim(states,df,pdf,normalize=False,extend=1.2):  
    smax=0
    for i in states:
        name,code=i
        state = df[(df.state==name) | (df.state==code)]
        s = state.select_dtypes(np.int).sum()[-1]
        if normalize:
            s = s/pdf[pdf.state==name].population.values[0]*100000
        if s > smax:
            smax=s   
    return smax*extend,log(smax*extend) 

def state(name='Massachusetts',code='MA',normalize_by_population=False,logy=False):
    state = df[(df.state==name) | (df.state==code)]
    s = state.select_dtypes(np.int).sum()
    if normalize_by_population:
        s = s/popusdf[popusdf.state==name].population.values[0]*100000
    opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,'padding':0.1}
    s.name = name
    lines = s.hvplot(**opts)
    dots = s.hvplot.scatter(**opts)
    hstate = lines * dots
    return hstate

sd='2020-02-28'
ed=(df.columns[-1]+dt.timedelta(1)).strftime('%Y-%m-%d')
xlim=(pd.date_range(sd,ed)[0],pd.date_range(sd,ed)[-1])

ulim,ulimlog=uplim(states,df,popusdf)
nulim,nulimlog=uplim(states,df,popusdf,True)

lo=[]
for normalize_by_population in [False,True]:
    for logscale in [False, True]:
    
        if normalize_by_population:
            ylim=(0.001,nulim)
            ylabel='Number of Deaths normalized by population [per 100,000]'
            title_format='U.S. COVID-19 Deaths: Normalized by Population, Status: {}'.format(df.columns[-1].date())
        else:
            ylim=(1,ulim)
            ylabel='U.S. Total Number of Deaths [Count]'
            title_format='U.S. COVID-19 Deaths: Total Count, Status: {}'.format(df.columns[-1].date())
        ol=[]
        for i in states:
            ol.append(state(*i,normalize_by_population,logscale))

        if logscale:
            title_format+=', Logarithmic Scale'

        overlay=hv.Overlay(ol)
        lo.append(overlay.opts(height=600,width=780,title_format=title_format, ylim=ylim,xlim=xlim,
                               ylabel=ylabel,xlabel='Date',xrotation=45,active_tools=['box_zoom'],toolbar='right'))
layout=hv.Layout(lo).opts(shared_axes=False,merge_tools=False)
layout.cols(1)

:Layout
   .Overlay.I   :Overlay
      .Curve.Massachusetts   :Curve   [index]   (Massachusetts)
      .Scatter.Massachusetts :Scatter   [index]   (Massachusetts)
      .Curve.Connecticut     :Curve   [index]   (Connecticut)
      .Scatter.Connecticut   :Scatter   [index]   (Connecticut)
      .Curve.Washington      :Curve   [index]   (Washington)
      .Scatter.Washington    :Scatter   [index]   (Washington)
      .Curve.Texas           :Curve   [index]   (Texas)
      .Scatter.Texas         :Scatter   [index]   (Texas)
      .Curve.California      :Curve   [index]   (California)
      .Scatter.California    :Scatter   [index]   (California)
      .Curve.New_York        :Curve   [index]   (New York)
      .Scatter.New_York      :Scatter   [index]   (New York)
      .Curve.Georgia         :Curve   [index]   (Georgia)
      .Scatter.Georgia       :Scatter   [index]   (Georgia)
      .Curve.Illinois        :Curve   [index]   (Illinois)
      .Scatter.Illinois      :Scatter   [index]   (Illinois)
      .Curve.Michigan        :Curve   [index]   (Michigan)
      .Scatter.Michigan      :Scatter   [index]   (Michigan)
   .Overlay.II  :Overlay
      .Curve.Massachusetts   :Curve   [index]   (Massachusetts)
      .Scatter.Massachusetts :Scatter   [index]   (Massachusetts)
      .Curve.Connecticut     :Curve   [index]   (Connecticut)
      .Scatter.Connecticut   :Scatter   [index]   (Connecticut)
      .Curve.Washington      :Curve   [index]   (Washington)
      .Scatter.Washington    :Scatter   [index]   (Washington)
      .Curve.Texas           :Curve   [index]   (Texas)
      .Scatter.Texas         :Scatter   [index]   (Texas)
      .Curve.California      :Curve   [index]   (California)
      .Scatter.California    :Scatter   [index]   (California)
      .Curve.New_York        :Curve   [index]   (New York)
      .Scatter.New_York      :Scatter   [index]   (New York)
      .Curve.Georgia         :Curve   [index]   (Georgia)
      .Scatter.Georgia       :Scatter   [index]   (Georgia)
      .Curve.Illinois        :Curve   [index]   (Illinois)
      .Scatter.Illinois      :Scatter   [index]   (Illinois)
      .Curve.Michigan        :Curve   [index]   (Michigan)
      .Scatter.Michigan      :Scatter   [index]   (Michigan)
   .Overlay.III :Overlay
      .Curve.Massachusetts   :Curve   [index]   (Massachusetts)
      .Scatter.Massachusetts :Scatter   [index]   (Massachusetts)
      .Curve.Connecticut     :Curve   [index]   (Connecticut)
      .Scatter.Connecticut   :Scatter   [index]   (Connecticut)
      .Curve.Washington      :Curve   [index]   (Washington)
      .Scatter.Washington    :Scatter   [index]   (Washington)
      .Curve.Texas           :Curve   [index]   (Texas)
      .Scatter.Texas         :Scatter   [index]   (Texas)
      .Curve.California      :Curve   [index]   (California)
      .Scatter.California    :Scatter   [index]   (California)
      .Curve.New_York        :Curve   [index]   (New York)
      .Scatter.New_York      :Scatter   [index]   (New York)
      .Curve.Georgia         :Curve   [index]   (Georgia)
      .Scatter.Georgia       :Scatter   [index]   (Georgia)
      .Curve.Illinois        :Curve   [index]   (Illinois)
      .Scatter.Illinois      :Scatter   [index]   (Illinois)
      .Curve.Michigan        :Curve   [index]   (Michigan)
      .Scatter.Michigan      :Scatter   [index]   (Michigan)
   .Overlay.IV  :Overlay
      .Curve.Massachusetts   :Curve   [index]   (Massachusetts)
      .Scatter.Massachusetts :Scatter   [index]   (Massachusetts)
      .Curve.Connecticut     :Curve   [index]   (Connecticut)
      .Scatter.Connecticut   :Scatter   [index]   (Connecticut)
      .Curve.Washington      :Curve   [index]   (Washington)
      .Scatter.Washington    :Scatter   [index]   (Washington)
      .Curve.Texas           :Curve   [index]   (Texas)
      .Scatter.Texas         :Scatter   [index]   (Texas)
      .Curve.California      :Curve   [index]   (California)
      .Scatter.California    :Scatter   [index]  

<a id='Mortality'></a>
## Mortality Rates Country Comparison
[back to top](#top)

The plot below shows the *mortality rate* in percent computed as:

$Mortality=\frac{Deaths}{Infected} * 100$

Two caveats:
- Because there is a timelag from infection to death, the rates computed on a daily basis might be underestimating the rate.
- The rates might be vastly overestimated because the lag in testing is widespread. South Korea has done the most intensive testing, hence South Korea data seem most reliable for mortality assessment.

In [65]:
countries=('China','Korea, South','Japan','Italy','Iran','Spain','Germany','Austria','France','US','Mexico','Canada','Brazil','India')

#Deaths
df_d = pd.read_csv(cv19_d)
df_d.columns = df_d.columns[0:4].append(pd.to_datetime(df_d.columns[4:]))
df_d.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df_d = df_d[[x in countries for x in df_d.country.values]]
# Infections confirmed cases
df_c = pd.read_csv(cv19_c)
df_c.columns = df_c.columns[0:4].append(pd.to_datetime(df_c.columns[4:]))
df_c.rename(columns={'Country/Region': 'country', 'Province/State': 'state',
                   'Lat': 'lat', 'Long': 'lon'}, inplace = True)
df_c = df_c[[x in countries for x in df_c.country.values]]


def uplim(df,pdf,normalize=False,extend=1.2):  
    smax=0
    for i in df.country:
        s = df[df.country==i].select_dtypes(np.int).sum()[-1]
        if normalize:
            s = s/(pdf[(pdf.country==i) & (pdf.Time==2019)].population.values[0]*10)
        if s > smax:
            smax=s   
    return smax*extend,log(smax*extend) 

def mortality(name='Germany',normalize_by_population=False,logy=False):
    country_deaths   = df_d[(df_d.country==name)]
    country_infected = df_c[(df_c.country==name)]
    d = country_deaths.select_dtypes(np.int).sum()
    i = country_infected.select_dtypes(np.int).sum()
    m=d/i*100
    if normalize_by_population:
        ulim,ulimlog=uplim(country,popdf2)
        m = m/(popdf2[(popdf2.country==name) & (popdf2.Time==2019)].population.values[0]*10)
    opts = {'legend': True, 'logy': logy, 'grid': True, 'width': 700, 'height': 400,'padding':0.1}
    m.name = name
    lines = m.hvplot(**opts)
    dots = m.hvplot.scatter(**opts)
    hstate = lines * dots
    return hstate

ylim=(-1,15)
ylabel='Mortality [%]'
title_format='COVID-19 Mortality (See Caveats!)'
normalize_by_population=False
logscale=False
ol=[]
for i in countries:
    ol.append(mortality(i,normalize_by_population,logscale))

overlay=hv.Overlay(ol).opts(height=600,width=780,title_format=title_format,ylabel=ylabel,
                            xlabel='Date',xrotation=45,ylim=ylim,logy=logscale,
                            active_tools=['box_zoom'],toolbar='right')
overlay

:Overlay
   .Curve.China               :Curve   [index]   (China)
   .Scatter.China             :Scatter   [index]   (China)
   .Curve.Korea_comma_South   :Curve   [index]   (Korea, South)
   .Scatter.Korea_comma_South :Scatter   [index]   (Korea, South)
   .Curve.Japan               :Curve   [index]   (Japan)
   .Scatter.Japan             :Scatter   [index]   (Japan)
   .Curve.Italy               :Curve   [index]   (Italy)
   .Scatter.Italy             :Scatter   [index]   (Italy)
   .Curve.Iran                :Curve   [index]   (Iran)
   .Scatter.Iran              :Scatter   [index]   (Iran)
   .Curve.Spain               :Curve   [index]   (Spain)
   .Scatter.Spain             :Scatter   [index]   (Spain)
   .Curve.Germany             :Curve   [index]   (Germany)
   .Scatter.Germany           :Scatter   [index]   (Germany)
   .Curve.Austria             :Curve   [index]   (Austria)
   .Scatter.Austria           :Scatter   [index]   (Austria)
   .Curve.France              :Curve   [index]   (France)
   .Scatter.France            :Scatter   [index]   (France)
   .Curve.US                  :Curve   [index]   (US)
   .Scatter.US                :Scatter   [index]   (US)
   .Curve.Mexico              :Curve   [index]   (Mexico)
   .Scatter.Mexico            :Scatter   [index]   (Mexico)
   .Curve.Canada              :Curve   [index]   (Canada)
   .Scatter.Canada            :Scatter   [index]   (Canada)
   .Curve.Brazil              :Curve   [index]   (Brazil)
   .Scatter.Brazil            :Scatter   [index]   (Brazil)
   .Curve.India               :Curve   [index]   (India)
   .Scatter.India             :Scatter   [index]   (India)

# We hope these data are informative and convey how serious we have to take the COVID-19 pandemic. Stay safe.